# Exercises in Fairness in Machine Learning

## Exercise 1

For this exercise, we will use the `adult` dataset (available on moodle or from the [UCI Machine Learning repository](https://archive.ics.uci.edu/dataset/2/adult)). Do the following:

1. Load in the dataset and correct the error in the income column (replace the "." with the empty string such that there are only two categories).
2. Create an X dataset using the variables "age", "workclass", "education", "occupation", "race", "sex", "hours-per-week". For the categorical variables with missing values, replace the missing values with a new category "Unknown". Also replace any values that are "?" with the value "Unknown (using `str.replace`, for instance)
3. Turn the five categorical variables in X into dummy variables and remove the original five variables (This will probably give you around 44 columns in X)
4. Create the response variable y, such that it is 1 if the `income` variable in the adult dataset is `>50K` and 0 if the value is `<=50K`.
5. Do a train-test split with 30% of the data for test (using `random_state=123`) and train a `XGBoost` classification model on the training data.
6. Evaluate your models using various evaluation metrics and look at the confusion matrix of your model.
7. To be able to calculate the various fairness metrics in regard to the variable `sex`, we need to construct two separate confusion matrices for the test dataset, one for `female` and one for `male`. First, create separate test sets for `female` and `male` as well as the predicted values for each gender. That is, create `X_test_female`, `X_test_male`, `y_test_female`, `y_test_male`, `y_pred_female`, and `y_pred_male`. (Hint: You can create `X_test_female` by `X_test_female = X_test[X_test["sex_Male"] == 0]` and `y_test_male` by `y_test_male = y_test[X_test["sex_Male"] == 1]`, for instance.)
8. Calculate the accuracy for female and male for the XGBoost model and comment on the results.
9. We can now create the True Positive (TP), True Negative (TN), False Positive (FP), and False Negative (FN) for each gender. That is, calculate the eight values `TP_f`, `TN_f`, `FP_f`, `FN_f`, `TP_m`, `TN_m`, `FP_m`, and `FN_m`. (Hint: You can calculate the False Positive for female (FP_f) by `FP_f = sum((y_test_female == 0) & (y_pred_female == 1))`.)
10. Is there error rate balance across different genders, i.e. are the false positive rate (FPR) and false negative rate (FNR) the same across the two genders?
11. Is there predictive parity across different genders?
12. Is there Statistical parity across different genders?
13. [Discussion question] Can your any of your models be used to make fair salary predictions?
14. [Discussion question] In what sense is the `adult` dataset biased (unfair)?
15. [Discussion question] If the dataset is biased, where could the bias potentially come from?
16. [Optional] If you balance the number of males and females in the dataset (like we balanced the response variable in the Churn example), will you model become more fair?

In [131]:
# 1. Load in the dataset and correct the error in the income column (replace the "." with the empty string such that there are only two categories).
import pandas as pd

# Indlæs datasættet
data = pd.read_csv("adult.csv")

# Vis de første rækker for at få overblik
data.head()


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [132]:
# Undersøg værdierne i 'income'-kolonnen
print(data['income'].unique())


['<=50K' '>50K' '<=50K.' '>50K.']


In [133]:
# Fjern mellemrum, punktummer og eventuelle andre tegn
data['income'] = data['income'].str.strip().str.replace('.', '', regex=False)


In [134]:
print(data['income'].unique())


['<=50K' '>50K']


 **2 Create an X dataset using the variables "age", "workclass", "education", "occupation", "race", "sex", "hours-per-week". For the categorical variables with missing values, replace the missing values with a new category "Unknown". Also replace any values that are "?" with the value "Unknown (using `str.replace`, for instance)**

In [135]:
# Step 1: select the specified columns
X = data[["age", "workclass","education","occupation","race","sex","hours-per-week"]]

# Step 2: Replace missing values and "?" with "Unknown"
categorical_columns={"workclass","education","occupation","race","sex"} 

for col in categorical_columns:
    # Replace missing values (NaN) with "Unknown"
    X[col].fillna("Unknown", inplace=True)

    # Replace "?" with "Unknown"
    X[col] = X[col].str.replace("?", "Unknown")

    
print(X.head())

   age         workclass  education         occupation   race     sex  \
0   39         State-gov  Bachelors       Adm-clerical  White    Male   
1   50  Self-emp-not-inc  Bachelors    Exec-managerial  White    Male   
2   38           Private    HS-grad  Handlers-cleaners  White    Male   
3   53           Private       11th  Handlers-cleaners  Black    Male   
4   28           Private  Bachelors     Prof-specialty  Black  Female   

   hours-per-week  
0              40  
1              13  
2              40  
3              40  
4              40  


C:\Users\Thoma\AppData\Local\Temp\ipykernel_7248\1284118882.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna("Unknown", inplace=True)
C:\Users\Thoma\AppData\Local\Temp\ipykernel_7248\1284118882.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col].fillna("Unknown", inplace=True)
C:\Users\Thoma\AppData\Local\Temp\ipykernel

In [136]:
# Step 3: Check for NaN values
print(X.isna().sum())


age               0
workclass         0
education         0
occupation        0
race              0
sex               0
hours-per-week    0
dtype: int64


In [137]:
print(X['workclass'].value_counts())
print(X['occupation'].value_counts())


workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
Unknown              2799
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64
occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Unknown              2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64


In [138]:
print((X == '?').sum())


age               0
workclass         0
education         0
occupation        0
race              0
sex               0
hours-per-week    0
dtype: int64


In [139]:
# 3 Turn the five categorical variables in X into dummy variables and remove the original five variables (This will probably give you around 44 columns in X)
X = pd.get_dummies(X, columns=['workclass', 'education', 'occupation', 'race', 'sex'], drop_first=True)

X.shape


(48842, 44)

In [140]:
print(X.columns)


Index(['age', 'hours-per-week', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Unknown', 'workclass_Without-pay', 'education_11th',
       'education_12th', 'education_1st-4th', 'education_5th-6th',
       'education_7th-8th', 'education_9th', 'education_Assoc-acdm',
       'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate',
       'education_HS-grad', 'education_Masters', 'education_Preschool',
       'education_Prof-school', 'education_Some-college',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',
       'occupation_Other-service', 'occupation_Priv-house-serv',
       'occupation_Prof-specialty', 'occupation_Protective-serv',
       'occupation_Sales', 'occupation_Tech-support',
       '

In [141]:
# 4 Create the response variable y, such that it is 1 if the `income` variable in the adult dataset is `>50K` and 0 if the value is `<=50K`.

# Opret y som en binær variabel
y = (data['income'] == '>50K').astype(int)


(data['income'] == '>50K') returnerer en boolesk serie (True eller False).

.astype(int) konverterer True til 1 og False til 0.

In [142]:
print(y.value_counts())
print(y.head())


income
0    37155
1    11687
Name: count, dtype: int64
0    0
1    0
2    0
3    0
4    0
Name: income, dtype: int32


In [143]:
# 5 Do a train-test split with 30% of the data for test (using `random_state=123`) and train a `XGBoost` classification model on the training data.

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# Train the model
model = xgb.XGBClassifier()
model.fit(X_train, y_train)


# Evaluate the model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.8195591346481949


In [144]:
# 6 Evaluate your models using various evaluation metrics and look at the confusion matrix of your model.
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay


Øverste række = Lav indkomst

Nederste række = Høj indkomst

Venstre side = Rigtige forudsigelser

Højre side = Forkerte forudsigelser

In [145]:

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Klassifikationsrapport (præcision, recall, F1-score)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))



Confusion Matrix:
[[10361   805]
 [ 1839  1648]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.93      0.89     11166
           1       0.67      0.47      0.55      3487

    accuracy                           0.82     14653
   macro avg       0.76      0.70      0.72     14653
weighted avg       0.81      0.82      0.81     14653



**Confusion Matrix** 

**10361** korrekt forudsagte med lav indkomst (0)

**805** personer med lav indkomst(0) som fejlagtigt blev forudsagt som høj indkomst

**1839** personer med høj indkomst (1) som som fejlagtigt blev forudsagt som lav indkomst

**1648** korrekt forudsagte personer med høj indkomst (1)

Modellen er god til at forudsige personer med lav indkomst (klasse 0) — 10361 korrekte ud af 11166.

Modellen har sværere ved at identificere personer med høj indkomst (klasse 1) — 1839 fejlklassifikationer ud af 3487.

**Precision (Præcision):** Hvor præcist modellen forudsiger hver klasse.

**Klasse 0:** 0.85 → Når modellen siger "lav indkomst", er den rigtig 85% af gangene.

**Klasse 1:** 0.67 → Når modellen siger "høj indkomst", er den kun rigtig 67% af gangene.

Høj præcision for klasse 0, men middel præcision for klasse 1.



**Recall (Genkaldelse):** Hvor godt modellen fanger de faktiske positive tilfælde.

**Klasse 0:** 0.93 → 93% af de personer, der faktisk har lav indkomst, blev korrekt identificeret.

**Klasse 1:** 0.47 → Kun 47% af de personer, der faktisk har høj indkomst, blev korrekt identificeret.

Recall for klasse 1 er lav — modellen misser mange personer med høj indkomst.

**F1-score:** En kombination af precision og recall (giver en samlet vurdering).

**Klasse 0:** 0.89 (God samlet præstation)

**Klasse 1:** 0.55 (Dårlig samlet præstation for høj indkomst)


In [146]:
# 7 To be able to calculate the various fairness metrics in regard to the variable sex, we need to construct two separate confusion matrices for the test dataset, one for female and one for male. First, create separate test sets for female and male as well as the predicted values for each gender. That is, create X_test_female, X_test_male, y_test_female, y_test_male, y_pred_female, and y_pred_male. (Hint: You can create X_test_female by X_test_female = X_test[X_test["sex_Male"] == 0] and y_test_male by y_test_male = y_test[X_test["sex_Male"] == 1], for instance.)

# Opret separate test-sæt for kvinder og mænd
X_test_female = X_test[X_test["sex_Male"] == 0]
X_test_male = X_test[X_test["sex_Male"] == 1]

# Opret de tilsvarende labels (y_test)
y_test_female = y_test[X_test["sex_Male"] == 0]
y_test_male = y_test[X_test["sex_Male"] == 1]


In [147]:
# Forudsigelser for kvinder
y_pred_female = model.predict(X_test_female)

# Forudsigelser for mænd
y_pred_male = model.predict(X_test_male)


Øverste række = Lav indkomst

Nederste række = Høj indkomst

Venstre side = Rigtige forudsigelser 

Højre side = Forkerte forudsigelser

In [148]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion Matrix for kvinder
print("Confusion Matrix - Female:")
print(confusion_matrix(y_test_female, y_pred_female))

# Confusion Matrix for mænd
print("\nConfusion Matrix - Male:")
print(confusion_matrix(y_test_male, y_pred_male))


Confusion Matrix - Female:
[[4303   67]
 [ 428   56]]

Confusion Matrix - Male:
[[6058  738]
 [1411 1592]]


In [149]:
# 8. Calculate the accuracy for female and male for the XGBoost model and comment on the results.


# Beregn accuracy
accuracy_female = accuracy_score(y_test_female, y_pred_female)
accuracy_male = accuracy_score(y_test_male, y_pred_male)

print(f"Accuracy for kvinder: {accuracy_female:.3f}")
print(f"Accuracy for mænd: {accuracy_male:.3f}")


Accuracy for kvinder: 0.898
Accuracy for mænd: 0.781


Modellen præsterer meget godt for kvinder, hvilket kan indikere, at data for denne gruppe er lettere for modellen at forstå — fx hvis der er færre ubalancer eller tydeligere mønstre.

Modellen præsterer væsentligt dårligere for mænd. Dette kan skyldes, at data for mænd er mere varieret eller at modellen har sværere ved at genkende mønstre blandt mænd.

Modellen klarer sig bedre på kvinder end på mænd, men det er vigtigt at huske, at recall og præcision for kvinder med høj indkomst tidligere var meget lav.

 **9. We can now create the True Positive (TP), True Negative (TN), False Positive (FP), and False Negative (FN) for each gender.** 
 
 That is, calculate the eight values `TP_f`, `TN_f`, `FP_f`, `FN_f`, `TP_m`, `TN_m`, `FP_m`, and `FN_m`.

  (Hint: You can calculate the False Positive for female (FP_f) by `FP_f = sum((y_test_female == 0) & (y_pred_female == 1))`.)

In [150]:
# True Positive (TP_f): korrekt forudsigelse af høj indkomst for kvinder
TP_f = sum((y_test_female == 1) & (y_pred_female == 1))

# True Negative (TN_f): korrekt forudsigelse af lav indkomst for kvinder
TN_f = sum((y_test_female == 0) & (y_pred_female == 0))

# False Positive (FP_f): forudsigelse af høj indkomst, når det i virkeligheden er lav indkomst
FP_f = sum((y_test_female == 0) & (y_pred_female == 1))

# False Negative (FN_f): forudsigelse af lav indkomst, når det i virkeligheden er høj indkomst
FN_f = sum((y_test_female == 1) & (y_pred_female == 0))


In [151]:
# True Positive (TP_m): korrekt forudsigelse af høj indkomst for mænd
TP_m = sum((y_test_male == 1) & (y_pred_male == 1))

# True Negative (TN_m): korrekt forudsigelse af lav indkomst for mænd
TN_m = sum((y_test_male == 0) & (y_pred_male == 0))

# False Positive (FP_m): forudsigelse af høj indkomst, når det i virkeligheden er lav indkomst
FP_m = sum((y_test_male == 0) & (y_pred_male == 1))

# False Negative (FN_m): forudsigelse af lav indkomst, når det i virkeligheden er høj indkomst
FN_m = sum((y_test_male == 1) & (y_pred_male == 0))


In [152]:
# Print resultater for kvinder (female)
print(f"TP_f: {TP_f}, TN_f: {TN_f}, FP_f: {FP_f}, FN_f: {FN_f}")

# Print resultater for mænd (male)
print(f"TP_m: {TP_m}, TN_m: {TN_m}, FP_m: {FP_m}, FN_m: {FN_m}")


TP_f: 56, TN_f: 4303, FP_f: 67, FN_f: 428
TP_m: 1592, TN_m: 6058, FP_m: 738, FN_m: 1411


In [153]:
# 10 Is there error rate balance across different genders, i.e. are the false positive rate (FPR) and false negative rate (FNR) the same across the two genders?

# Beregn confusion matrix for kvinder
cm_female = confusion_matrix(y_test_female, y_pred_female)

# Beregn FPR og FNR for kvinder
FPR_f = cm_female[0, 1] / (cm_female[0, 1] + cm_female[1, 1])  # FP / (FP + TN)
FNR_f = cm_female[1, 0] / (cm_female[1, 0] + cm_female[0, 0])  # FN / (FN + TP)

# Beregn confusion matrix for mænd
cm_male = confusion_matrix(y_test_male, y_pred_male)

# Beregn FPR og FNR for mænd
FPR_m = cm_male[0, 1] / (cm_male[0, 1] + cm_male[1, 1])  # FP / (FP + TN)
FNR_m = cm_male[1, 0] / (cm_male[1, 0] + cm_male[0, 0])  # FN / (FN + TP)

# Udskriv resultaterne
print(f"Female - FPR: {FPR_f:.3f}, FNR: {FNR_f:.3f}")
print(f"Male - FPR: {FPR_m:.3f}, FNR: {FNR_m:.3f}")


Female - FPR: 0.545, FNR: 0.090
Male - FPR: 0.317, FNR: 0.189


There is an imbalance in the error rates between genders:

The False Positive Rate (FPR) is higher for women (0.545), meaning that 54.5% of women who actually have low income are incorrectly predicted as having high income. In contrast, the FPR for men (0.317) is lower, with 31.7% of men who actually have low income being wrongly predicted as high income. This suggests that the model is more prone to incorrectly classifying women as having high income, which could indicate a potential bias against women in predicting low income.

On the other hand, the False Negative Rate (FNR) is lower for women (0.090), meaning that only 9% of women with high income are incorrectly predicted as having low income. For men, the FNR is higher (0.189), with 18.9% of men with high income being wrongly predicted as having low income. This shows that the model is more accurate at identifying women with high income compared to men.

In [ ]:
#11 Is there predictive parity across different genders?

# Calculate Positive Prediction Rate (PPR) for females
PPR_f = (TP_f + FP_f) / (TP_f + TN_f + FP_f + FN_f)

# Calculate Positive Prediction Rate (PPR) for males
PPR_m = (TP_m + FP_m) / (TP_m + TN_m + FP_m + FN_m)

# Print the results
print(f"Female - PPR: {PPR_f:.3f}")
print(f"Male - PPR: {PPR_m:.3f}")


Female - PPR: 0.025
Male - PPR: 0.238


PPR for females is 0.025, meaning only 2.5% of the total predictions made for females were predicted as high income (positive class).

PPR for males is 0.238, meaning 23.8% of the total predictions made for males were predicted as high income.

Conclusion:

There is a imbalance in predictive parity between genders. The model is making positive predictions (high income) much more frequently for males (23.8%) than for females (2.5%).

This indicates that the model's predictions are not equally distributed between genders, and there may be a bias in the model's predictions favoring males over females.



**12 Is there Statistical parity across different genders?**

Statistical Parity requires that PPR for females and PPR for males should be the same or at least close to each other.

There is no statistical parity across genders since the positive prediction rates for women (2.5%) and men (23.8%) differ.

 This suggests a bias in the model where it is making a lot more positive predictions for men than for women.

**13 [Discussion question] Can any of your models be used to make fair salary predictions?** 

Based on our results, the model shows no statistical parity (significant difference in Positive Prediction Rate between males and females), meaning there is no balance in how high income is predicted across genders. FPR is higher for females and FNR is lower for females, indicating bias in predictions, as fewer women are predicted to have high income.

To achieve fair salary predictions, the model should:

Avoid bias in the data, as the results show an imbalance in predictions between genders.
Work towards statistical parity and predictive parity so that both males and females receive fair predictions.
Further adjustments to the model are needed to ensure fairness in salary predictions.

 **14  [Discussion question] In what sense is the `adult` dataset biased (unfair)?**
 
The Adult dataset is biased because it reflects existing inequalities in society. There are historical wage gaps between men and women, meaning men often earn higher salaries, even with similar qualifications. Additionally, there can be differences in wages and job opportunities based on race, which creates bias in the dataset. Disparities in access to education and career opportunities also contribute to bias, as they affect income levels depending on one's background. Overall, the Adult dataset is biased because it mirrors societal inequalities, which can lead to unfair predictions if models are trained on it.

**15 [Discussion question] If the dataset is biased, where could the bias potentially come from?**

The bias in the Adult dataset could possiblecome from existing societal inequalities, such as wage gaps and discrimination based on gender and race. 
There may also be sampling bias if certain groups are underrepresented, feature bias from using characteristics like gender or race, and data collection bias if certain groups are favored in the data.

**16 [Optional] If you balance the number of males and females in the dataset (like we balanced the response variable in the Churn example), will you model become more fair?**

Balancing the number of males and females in the dataset might help reduce bias related to representation, but it doesn't guarantee fairness. 
model could still be biased if it relies on unfair features like gender or race.